## Text to speech
Try text to text-to-speech Watson Service

You need to replace the next cell with the proper Cloud Object Storage (COS) credentials.<br/>
The easiest is to simply insert the credentials from an existing file.

In [ ]:
# COS credentials
# The easiest would be to get them from an existing file

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials = {
    'IAM_SERVICE_ID': 'iam-ServiceId-65647f57-cd61-4f37-abd9-38b10084d177',
    'IBM_API_KEY_ID': '9OBEPHS0jj5q0FdEFWpF-US8qWWwiqFtRkeH6njgVaar',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',
    'BUCKET': 'bscstesting-donotdelete-pr-paqxy5fmsmaykn'
}


In [ ]:
# IBM COS interface
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# need to figure out the bucket by looking at the service
# the bucket name starts with the project name

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials["IBM_API_KEY_ID"],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

Bucket=credentials["BUCKET"]

## Install the Watsn API

In [ ]:
!pip install --upgrade "ibm-watson>=3.0.4"

## Getting a language translator connection
Once the text to voice service is created, make sure to create the credentials and insert the values in the next cell.

* Text-to-speech documentation: https://console.bluemix.net/docs/services/text-to-speech
* Api reference: https://www.ibm.com/watson/developercloud/text-to-speech/api

In [ ]:
t2s_credentials = {
  "apikey": "2pNxBu3UVaPe402-bBnMoLJtsevhqGMNw7Uu5TQaHaLx",
  "iam_apikey_description": "Auto-generated for key 2301ef1f-27eb-4dd7-b9aa-454d25c71b76",
  "iam_apikey_name": "wdp-writer",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/e46675b7f1bf89b09b5badfb3bd4a7b5::serviceid:ServiceId-bae88031-022b-4e95-8539-fc0e9ff1b8a9",
  "url": "https://stream.watsonplatform.net/text-to-speech/api"
}

In [ ]:
# create the basic header
#import urllib3, requests, json

#t2s_headers = urllib3.util.make_headers(
#    basic_auth='{username}:{password}'.format(username=t2s_credentials['username'], 
#                                              password=t2s_credentials['password'])
#)
import json
from ibm_watson import TextToSpeechV1

text2speech = TextToSpeechV1(
    iam_apikey=t2s_credentials['apikey'],
    url=t2s_credentials['url']
)


## Available voices
Watson text to voice has multiple voices in multiple languages.<br/>
The next cell displays all the available voices. Make sure to pick on that fits the language of the text.

In [ ]:
voices = text2speech.list_voices().get_result()
print(json.dumps(voices, indent=2))

In [ ]:
t2s_url = '{}/v1/voices'.format(t2s_credentials['url'])
t2s_response = requests.get(t2s_url, headers=t2s_headers)
# mltoken = json.loads(t2s_response.text).get('token')

# header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
t2s_doc = json.loads(t2s_response.text)
print (json.dumps(t2s_doc, indent=2, sort_keys=True))

## Convert some text

In [ ]:
InputText="The quick brown fox jumps over the lazy dog."

## Query parameters
Accept: type of output<br/>
voice: voice used. default en-US_MichaelVoice

Other english voice: en-US_AllisonVoice, en-US_LisaVoice, en-GB_KateVoice

In [ ]:
t2s_response = text2speech.synthesize(InputText,
                           voice='en-US_AllisonVoice',
                           accept='audio/mp3')
print(t2s_response.status_code) # Should be 200

## Write the local file to COS

In [ ]:
import io
cos.upload_fileobj(io.BytesIO(t2s_response.get_result().content), Bucket, 'voice.mp3')

## Convert speech to text
This assumes that you have created a watson service and have a copy of the credentials<br/>
First create the credentials for the watson service
* Documentation: https://console.bluemix.net/docs/services/speech-to-text
* API reference: https://www.ibm.com/watson/developercloud/speech-to-text/api/v1/

In [ ]:
# speech to text credentials
s2t_credentials = {
  "apikey": "uiBmgYn0wa905P9RgGx-hQKYUlUxap71mJPRTn3BVZ7M",
  "iam_apikey_description": "Auto-generated for key 47d46c1e-e05b-4217-ab39-f62fb5bc74a6",
  "iam_apikey_name": "wdp-writer",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/e46675b7f1bf89b09b5badfb3bd4a7b5::serviceid:ServiceId-c1ba18f6-3f21-417e-b0c7-c901562be362",
  "url": "https://stream.watsonplatform.net/speech-to-text/api"
}

## Find out which models are available

In [ ]:
from ibm_watson import SpeechToTextV1
import json

speech2text = SpeechToTextV1(
    iam_apikey=s2t_credentials['apikey'],
    url=s2t_credentials['url'])


In [ ]:
speech_models = speech2text.list_models().get_result()
print(json.dumps(speech_models, indent=2, sort_keys=True))

## Convert speech to text
Convert the output of text-to-speech back to text<br/>
Default model: en-US_BroadbandModel

The query for a local file would be:<br/>
`s2t_response = requests.post(s2t_url2, files={'file': open('./voice.mp3', 'rb')}, headers=s2t_headers2)`

The query for binary data such as the one generated form the text-to-speech content would be:<br/>
`s2t_response = requests.post(s2t_url2, files={'file': io.BytesIO(t2s_response.content)}, headers=s2t_headers2)`

Instead, we are reading the `voice.mp3` that we wrote to the IBM Cloud Object Storage (COS) in the earlier step.

In [ ]:
# from os.path import join, dirname
voice_input = cos.get_object(Bucket=Bucket,Key='voice.mp3')['Body']

s2t_response = speech2text.recognize(
        audio=io.BytesIO(voice_input.read()),
        content_type='audio/mp3')
print(s2t_response.status_code) # Should be 200
print(json.dumps(s2t_response.get_result(), indent=2))

In [ ]:
result = s2t_response.get_result()
print(result['results'][0]['alternatives'][0]['transcript'])